<center>
<img src="./images/EAN.jpg" style="width:1200px">
</center>

<center>
<img src="./images/0_intro_ml.jpg" style="width:1200px">
</center>

# Algorithm Chains and Pipelines


## Instructors:

>Leonardo A. Espinosa, PhD. Instructor.
(*email*: laespinosa@universidadean.edu.co)

> Ruben D. Acosta, MSc. Instructor.
(*email*:  rdacostav@universidadean.edu.co)

1\. <a href="#/25/1">Algorithm Chains and Pipelines</a>

   * <a href="#/31/1">The *Pipeline* Class</a>.
   * <a href="#/33/1">An example of Information Leakage</a>.
   * <a href="#/35/1">The General Pipeline Interface</a>
   
2\. <a href="#/42/1">Conclusions</a>

# Algorithm Chains and Pipelines

## Pipeline and GridSearchCV Classes

A machine learning problem usually includes:

* Scaling the data and combining features by hand. Learning features using unsupervised machine learning.

* Chaining together many different processing steps and machine learning models.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mglearn
from IPython.display import display

from matplotlib import rc
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 25}
rc('font', **font) 

plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['lines.linewidth'] = 5.0
plt.rcParams['lines.markersize'] = 15.0

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# load and split the data
cancer = load_breast_cancer() 
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

# compute minimum and maximum on the training data
scaler = MinMaxScaler().fit(X_train)

# rescale the training data
X_train_scaled = scaler.transform(X_train)
svm = SVC()

# learn an SVM on the scaled training data
svm.fit(X_train_scaled, y_train)
#svm.fit(X_train, y_train)

# scale the test data and score the scaled data
X_test_scaled = scaler.transform(X_test)
print("Test score: {:.2f}".format(svm.score(X_test_scaled, y_test)))

## Parameter Selection with Preprocessing

* Find the best parameters for *SVC* using *GridSearch* $\to$ Just add the grid parameters.


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Best set score: {:.2f}".format(grid.score(X_test_scaled, y_test)))
print("Best parameters: ", grid.best_params_)

# ERROR $\Rightarrow$ The grid search is done using the scaled data!
* But the scaled data was done on the whole data in the training set.
* For each split in the cross-validation some information is leaked from the test set into the training set.

<center><img src="./images/04-pipelines.png" alt="Drawing" style="width:1500px" align="middle"/></center>




# The *Pipeline* Class

You can create a *workflow* for training and classification after preprocessing.

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])
pipe.fit(X_train, y_train)
print("Test score: {:.2f}".format(pipe.score(X_test, y_test)))


## Pipelines in Grid Searches
Pipelines and grid search are combined in the same way that any estimator.


In [ ]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
               'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

# Information Leakage

Example adapted from *The Elements of Statistical Learning*.

In [ ]:
rnd = np.random.RandomState(seed=0)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_regression
select = SelectPercentile(score_func=f_regression, percentile=5).fit(X, y)
X_selected = select.transform(X)
print("X_selected.shape: {}".format(X_selected.shape))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
print("Cross-validation accuracy (cv only on ridge): {:.2f}".format(
    np.mean(cross_val_score(Ridge(), X_selected, y, cv=5))))

$R^2$ computed by cross-validation is 0.91 $\to$ a very good model. **ERROR**

## Proper model with cross-validation using a pipeline

In [ ]:
pipe = Pipeline([("select", SelectPercentile(score_func=f_regression, 
                                             percentile=5)),
               ("ridge", Ridge())])
print("Cross-validation accuracy (pipeline): {:.2f}".format(
np.mean(cross_val_score(pipe, X, y, cv=5))))

a negative $R^2$ score, indicating a very poor model.

## Pipeline Creation with make_pipeline

In [ ]:
from sklearn.pipeline import make_pipeline
# standard syntax
pipe_long = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC(C=100))])
# abbreviated syntax
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))

print("Pipeline steps:\n{}".format(pipe_short.steps))

In general, the step names are just lowercase versions of the class names.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), StandardScaler())
print("Pipeline steps:\n{}".format(pipe.steps))

**Note:** In this case it might be better to use the Pipeline construction with explicit name.

## Grid-Searching Preprocessing Steps and Model Parameters

In [ ]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target,random_state=0)

pipe = make_pipeline(StandardScaler(),PolynomialFeatures(),Ridge())

In [ ]:
param_grid = {'polynomialfeatures__degree': [1, 2, 3],
              'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(12,8))
plt.matshow(grid.cv_results_['mean_test_score'].reshape(3, -1),
            vmin=0, cmap="viridis")
plt.xlabel("ridge__alpha")
plt.ylabel("polynomialfeatures__degree")
plt.xticks(range(len(param_grid['ridge__alpha'])), param_grid['ridge__alpha'])
plt.yticks(range(len(param_grid['polynomialfeatures__degree'])),
param_grid['polynomialfeatures__degree'])
plt.colorbar()

In [ ]:
print("Best parameters: {}".format(grid.best_params_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

## Exhaustive Grid Search
* Again, it is not a recomendable strategy.
* If your dataset is large you can try on a random subset.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])

param_grid = [
    {'classifier': [SVC()], 'preprocessing': [StandardScaler(), None],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestClassifier(n_estimators=100)],
     'preprocessing': [None], 'classifier__max_features': [1, 2, 3]}]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

## Randomized Parameter Optimization

`RandomizedSearchCV` implements a randomized search over parameters, where each setting is sampled from a distribution over possible parameter values.

Advantages:
* A budget can be chosen independent of the number of parameters and possible values.
* Adding parameters that do not influence the performance does not decrease efficiency.

See: Bergstra, J. and Bengio, Y., Random search for hyper-parameter optimization, The Journal of Machine Learning Research (2012).

In [ ]:
from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier

# get some data
X, y = load_digits(return_X_y=True)

# build a classifier
clf = SGDClassifier(loss='hinge', penalty='elasticnet',fit_intercept=True)

In [ ]:
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
# specify parameters and distributions to sample from
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [ ]:
# use a full grid over all parameters
param_grid = {'average': [True, False],
              'l1_ratio': np.linspace(0, 1, num=10),
              'alpha': np.power(10, np.arange(-4, 1, dtype=float))}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

## GridSearchCV 2.0 -- New and Improved

Cutting edge hyperparameter tuning techniques (bayesian optimization, early stopping, distributed execution) can provide significant speedups over grid search and random search.

<center>
<img src="./images/tune+sklearn.png" style="width:800px">
    https://github.com/ray-project/tune-sklearn
</center>

### Tune: Scalable Hyperparameter Tuning

Tune is a Python library for experiment execution and hyperparameter tuning at any scale (https://docs.ray.io/en/latest/tune.html). Core features:

* Launch a multi-node distributed hyperparameter sweep in less than 10 lines of code.

* Supports any machine learning framework, including PyTorch, XGBoost, MXNet, and Keras.

* Natively integrates with optimization libraries such as HyperOpt, Bayesian Optimization, and Facebook Ax.

* Choose among scalable algorithms such as Population Based Training (PBT), Vizier’s Median Stopping Rule, HyperBand/ASHA.

* Visualize results with TensorBoard.

* Move your models from training to serving on the same infrastructure with Ray Serve.

Here’s what tune-sklearn has to offer:

* Consistency with Scikit-Learn API: tune-sklearn is a drop-in replacement for GridSearchCV and RandomizedSearchCV, so you only need to change less than 5 lines in a standard Scikit-Learn script to use the API.
* Modern hyperparameter tuning techniques: tune-sklearn is the only Scikit-Learn interface that allows you to easily leverage Bayesian Optimization, HyperBand, and other optimization techniques by simply toggling a few parameters.
* Framework support: tune-sklearn is used primarily for tuning Scikit-Learn models, but it also supports and provides examples for many other frameworks with Scikit-Learn wrappers such as Skorch (Pytorch), KerasClassifiers (Keras), and XGBoostClassifiers (XGBoost).
* Scale up: Tune-sklearn leverages Ray Tune, a library for distributed hyperparameter tuning, to efficiently and transparently parallelize cross validation on multiple cores and even multiple machines.

<center>
<img src="./images/00_hands-on.jpg" style="width:1200px">
</center>

### Exercise

Here we explore the pipeline class.

* Create a pipeline class combining one scaler and one model.
* Run a gridsearch with this pipeline on your dataset or the wine dataset.

# Conclusions
* Split into training data for model building, validation data for model and parameter selection, and test data for model evaluation.
* The Pipeline class is a general-purpose tool to chain together multiple processing steps in a machine learning workflow.
* Choosing the right combination of feature extraction, preprocessing, and models is somewhat of an art, and often requires some trial and error.

<center>
<img src="./images/00_questions.jpg" style="width:1200px">
</center>

## Tips for parameter search

* **Specifying an objective metric**. For some applications, other scoring functions are better suited
* **Specifying multiple metrics for evaluation**. `GridSearchCV` and `RandomizedSearchCV` allow specifying multiple metrics for the scoring parameter.
* **Composite estimators and parameter spaces**. Use pipelines and caching transformers to avoid repeated computation. See `sklearn` manual.
* **Parallelism**.
* **Robustness to failure**. Setting `error_score=0` (or `=np.NaN`) will make the procedure robust to such failure.

## Alternatives to brute force parameter search

* **Model specific cross-validation**. Some models can fit data for a range of values of some parameter almost as efficiently as fitting the estimator for a single value of the parameter. This feature can be leveraged to perform a more efficient cross-validation used for model selection of this parameter. eg. `linear_model.RidgeCV`, `linear_model.LogisticRegressionCV`.
* ** Information Criterion**. Some models can offer an information-theoretic closed-form formula of the optimal estimate of the regularization parameter by computing a single regularization path (instead of several when using cross-validation). eg. `linear_model.LassoLarsIC`
* **Out of Bag Estimates**. When using ensemble methods base upon bagging, i.e. generating new training sets using sampling with replacement, part of the training set remains unused. For each classifier in the ensemble, a different part of the training set is left out. eg. `ensemble.RandomForestClassifier`, `ensemble.GradientBoostingClassifier`




<center>
<img src="./images/00_questions.jpg" style="width:1200px">
</center>

<center>
<img src="./images/00_thats_all.jpg" style="width:1200px">
</center>